## CMPE 462 HOMEWORK 1
Due Date: 21.03.2019

In [1]:
# Allowed libraries
import numpy as np
import matplotlib.pyplot as plt
# For loading data, you can import other libraries, but further usage is not allowed.

### 1.) Review bias/variance dilemma.
Replicate the bias-variance-plot (bias,
variance and error vs. model complexity) given in Figure 4.6 in the textbook
or in lecture notes [W04.pdf, pg.52]. For this:

* Generate 20 values {$x^t$} from the range [0, 5] from uniform distribution.  
* Generate 100 different samples $X_s$. Each sample contains 20 instances {$x^t, y^t_s$} where $y^t_s = f(x^t) + \epsilon$, $f(x) =2× sin(1.5 × x)$, and $ \epsilon ∼ N (0, 1)$.
* For each sample, fit polynomial models of order 1,3, and 5.
* Plot bias, variance and error of these models.

In [8]:
SIZE = 20
SAMPLE_COUNT = 100

In [9]:
# Solution Here
x_values = np.random.uniform(low=0,high=5,size=SIZE)

In [12]:
def calc_y(x):
    return 2 * np.sin(1.5 * x)

In [14]:
# Generate 100 different samples.
samples = []
for i in range(SAMPLE_COUNT):
    sample = []
    for x_value in x_values:
        sample.append((x_value,calc_y(x_value) + np.random.normal(loc=0,scale=1)))
    samples.append(sample)


In [18]:
samples[2]

[(2.7718741090896821, -0.82413176775662411),
 (3.834729525969145, -0.70190328316160788),
 (3.4333070679590953, -1.6165967142327755),
 (0.63655007543930497, 0.29469827691122985),
 (4.8464562437980838, 1.4601126392024397),
 (1.1569890272293497, 1.9629207564142488),
 (4.5675068640726826, 1.6683746359428797),
 (0.56429679787658726, 0.38697016846876608),
 (4.8890777644327281, 0.98026196172197944),
 (3.6813519302760622, -2.0263363186846872),
 (0.017299436220963305, 0.029392164519705243),
 (4.0722250246694189, -0.62912499087500984),
 (0.88921719077492911, 1.9195381900254589),
 (0.10919031351348363, 1.1014646397993486),
 (2.6009354551767565, 0.028872017772727743),
 (2.6069438343243183, -3.443719784199486),
 (1.1968145335971476, 2.5317086204326871),
 (1.7601006454529307, 1.071079311427374),
 (3.2676912616416756, -2.5995374492409624),
 (2.4695346870838115, -2.3115823941153253)]

### 2.) Review cross-validation.
Replicate the cross-validation error plot (training and validation error vs. model complexity plot) given in Figure 4.7 in the textbook or in lecture notes [W04.pdf, pg52]. For this:
* Generate 10 samples containing 100 instances as in part 1.
* Split each sample to training and validation sets, fit polynomial models of order 1,3, and 5.
* Plot mean training and validation error (mean square error) of each model.

In [ ]:
# Solution Here

### 3.) Use of real dataset. 
Download Iris Data Set from UCI Machine Learning Repository. Randomly split it into training and test sets.
* For each feature, plot histogram of classes. For better visualiziation, plot classes in same plot but with different colors. (For this you can use plt.subplots)
* Consider only two classes: Iris Setosa and Iris Versicolour.
* Assuming Gaussian distribution, apply parametric classification, and find which particular feature (sepal length, sepal width, petal length, petal width) is most successful in classifying instances.
    - For this, treat each feature separately. find maximum likelihood estimate of the parameters $(μ, σ^2)$ of different classes using training set. (You can use plt.errorbar for visualization, use one column for each feature and visualize both classes in same plot using different colors).
    - Using these parameters, predict the classes of the instances in the test set, and calculate the error.

In [ ]:
# Solution Here